### Imports

In [11]:
import os, sys
from bs4 import BeautifulSoup
import pandas as pd
import requests

### NASA Mars News

In [69]:
# request url for mars news api
json_url=('https://mars.nasa.gov/api/v1/news_items/'
          '?page=0&per_page=10'
          '&order=publish_date+desc%2Ccreated_at+desc'
          '&search='
          '&category=19%2C165%2C184%2C204'
          '&blank_scope=Latest')

response = requests.get(json_url)
if response.status_code == 200:
    a=response.json()
    news_item = a.get('items')[0]
    news_title = news_item.get('title')
    news_description = news_item.get('description')
    news_p = BeautifulSoup(news_item.get('body'), 'html.parser').p.get_text()
    pass

In [70]:
print("Title:", news_title, end="\n\n")
print("Description:", news_description, end="\n\n")
print("Blurb:", news_p, end="\n\n")

Title: NASA's Mars 2020 Comes Full Circle

Description: Aiming to pinpoint the Martian vehicle's center of gravity, engineers took NASA's 2,300-pound Mars 2020 rover for a spin in the clean room at JPL. 

Blurb: ​Engineers took NASA's Mars 2020 for a spin on Aug. 29, 2019. The 2,300-pound (1,040-kilogram) Martian vehicle was rotated clockwise and counterclockwise at about 1 revolution per minute on what is called a spin table in the clean room of the Spacecraft Assembly Facility at NASA's Jet Propulsion Laboratory in Pasadena, California. (The rotation was speeded up in the video above.) The engineers were looking for the rover's center of gravity, or the point at which weight is evenly dispersed on all sides.



### JPL Mars Space Images - Featured Image

In [81]:
url=('https://www.jpl.nasa.gov/spaceimages/'
     '?search='
     '&category=Mars')    

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser').\
        find('article', 'carousel_item')
    featured_image_url = 'https://www.jpl.nasa.gov' + soup.attrs['style'].split("'")[1]
    pass

In [82]:
print("Image URL:", featured_image_url)

Image URL: https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA02570-1920x1200.jpg


### Mars Weather

In [ ]:
url=('https://twitter.com/marswxreport'
     '?lang=en')


### Mars Hemispheres

In [5]:
url=('https://astrogeology.usgs.gov/search/results'
     '?q=hemisphere+enhanced'
     '&k1=target'
     '&v1=Mars')


